In [ ]:
# para extraer mfcc (características del audio)
!pip install python_speech_features

In [ ]:
# para transformar de mp3 a wav
!pip install pydub

In [3]:
import numpy as np
import os
import pickle
import random
import pandas as pd
import sklearn
import scipy.io.wavfile as wav
from os import path
from pydub import AudioSegment
from python_speech_features import mfcc
from sklearn.metrics import classification_report
from collections import defaultdict

In [ ]:
# dataset
!wget https://github.com/abcom-mltutorials/music-genre/archive/master.zip

In [ ]:
!unzip "/content/master.zip"

In [6]:
directory = "/content/music-genre-master/Dataset"
filelist=[]
for path, subdirs, files in os.walk(directory):
   for file in files:
       if (file.endswith('.wav') or file.endswith('.WAV')):
           filelist.append(os.path.join(path, file))
number_of_files=len(filelist)
print(number_of_files)

900


In [7]:
def feature_extraction(file):
 features=[]
 (sampleRate,data) = wav.read(file)
 mfcc_feature = mfcc(data,sampleRate,
                           winlen=0.020,
                           appendEnergy = False)
 meanMatrix = mfcc_feature.mean(0)
 for x in meanMatrix:
   features.append(x)
 return features

In [8]:
datasetDirectory = "/content/music-genre-master/Dataset/"
 
featureSet=[]
i=0
for folder in os.listdir(datasetDirectory):
   i+=1
   if i > 9: # the number of genres is 9
       break  
   for files in os.listdir(datasetDirectory+folder):
     x=datasetDirectory+folder+"/"+files
     features=feature_extraction(x)
     j=0
     for x in features:
       featureSet.append(x)
       j=j+1
       if(j%13==0):
         featureSet.append(i) 

In [9]:
df = pd.DataFrame(columns=['m1','m2','m3','m4','m5','m6','m7',
                          'm8','m9','m10','m11','m12','m13','target'])

In [10]:
i=1
n=[]
for j in featureSet:
 n.append(j)
 #13 features + 1 taget
 if(i%14==0):
   df = df.append({'m1':n[0],'m2':n[1],'m3':n[2],'m4':n[3],'m5':n[4],
                   'm6':n[5],'m7':n[6],'m8':n[7],'m9':n[8],'m10':n[9],
                   'm11':n[10],'m12':n[11],'m13':n[12],'target':n[13]},
                  ignore_index=True)
   n=[]
 i=i+1

In [11]:
x1=df[['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12','m13']]
x1.shape

Y = df[['target']]
Y.shape

(900, 1)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x1, Y,
                                                   test_size=0.2,
                                                   random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train,y_train)

Ejemplo con wav

In [14]:
audio_file="/content/music-genre-master/new_audio_file.wav"
audio_feature=feature_extraction(audio_file)

In [15]:
from collections import defaultdict
results=defaultdict(int)
i=1
for folder in os.listdir("/content/music-genre-master/Dataset/"):
    results[i]=folder
    i+=1
pred_audio=clf.predict([audio_feature])
results[int(pred_audio)]

'rock'

In [24]:
results

defaultdict(int,
            {1: 'blues',
             2: 'hiphop',
             3: 'metal',
             4: 'reggae',
             5: 'classical',
             6: 'country',
             7: 'disco',
             8: 'rock',
             9: 'pop'})

Ejemplo con mp3

In [ ]:
!wget https://raw.githubusercontent.com/abcom-mltutorials/music/master/bhatiyar.mp3

In [17]:
src = "/content/bhatiyar.mp3"
dst = "test.wav"

In [ ]:
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

In [ ]:
audio_file="/content/test.wav"
audio_feature=feature_extraction(audio_file)

In [20]:
pred_audio=clf.predict([audio_feature])
results[int(pred_audio)-2]

'reggae'

Obtener modelo

In [21]:
import joblib

In [ ]:
joblib.dump(clf, 'music_model.joblib')

Créditos a: http://education.abcom.com/music-genre-classification/